In [5]:
import pandas as pd 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score

In [15]:
data = pd.read_csv("data/train.csv")
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [16]:
import random
 
sentiment_data = list(zip(data["id"], data["comment_text"], data["toxic"]))
#random.shuffle(sentiment_data)
 
# 80% for training
train_id, train_X, train_y = zip(*sentiment_data[:130000])
 
# Keep 20% for testing
test_id, test_X, test_y = zip(*sentiment_data[130000:])

In [24]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> tokenizers
Command 'tokenizers' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


KeyboardInterrupt: 

In [36]:
from nltk import word_tokenize
from nltk.sentiment.util import mark_negation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin
 
 
clf = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="word",
                                   #tokenizer=word_tokenize,         # ! Comment line to include mark_negation and uncomment next line
                                   tokenizer=lambda text: mark_negation(word_tokenize(text)), 
                                   preprocessor=lambda text: text.replace("<br />", " "),)),
    ('classifier', LinearSVC())
])

clf.fit(train_X, train_y)
clf.score(test_X, test_y)

0.95532785499306749

In [45]:
#%%time
unigram_bigram_clf = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="word",
                                   ngram_range=(1, 2),
                                   tokenizer=word_tokenize,
                                   #tokenizer=lambda text: mark_negation(word_tokenize(text)),
                                   preprocessor=lambda text: text.replace("<br />", " "),
                                   max_df=0.5) ),
    ('classifier', LinearSVC())
])
 
unigram_bigram_clf.fit(train_X, train_y)
unigram_bigram_clf.score(test_X, test_y)

0.95698488383889624

In [ ]:
" "),
                                   max_features=10000) ),

In [ ]:
" "),)),

In [ ]:
#top (max_df=0.5?)=0.95705251 melkeen sama mark_negation 0.956